In [1]:
%reload_ext dotenv
%dotenv ../.env

In [2]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [3]:
import pandas as pd

In [4]:
from google import genai

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works in a few words"
)
print(response.text)

Learns from data to recognize patterns and make decisions.


In [ ]:
import json

from strands_solver.word_finder import WordFinder

# Load puzzle from JSON
with open("../puzzles/2025-09-17.json", "r") as f:
    puzzle_data = json.load(f)

theme = puzzle_data["theme"]
grid = puzzle_data["grid"]

print(f"Theme: {theme}")
print(f"Grid size: {len(grid)}x{len(grid[0])}")

# Find all words in the grid
finder = WordFinder(grid)
found_strands = finder.find_all_words()

# Create list with theme first, then all found words
words = [theme] + list({strand.string for strand in found_strands})
print(f"\nTotal items to embed: {len(words)}")

Theme: We beg to differ
Grid size: 8x6

Total items to embed: 508


In [ ]:
"ATTRACT" in words

True

In [ ]:
from google import genai
from google.genai.types import EmbedContentConfig
from tenacity import retry, stop_after_attempt, wait_fixed

client = genai.Client()


# Retry on any exception, wait 60 seconds between retries, max 5 attempts
@retry(
    stop=stop_after_attempt(5),
    wait=wait_fixed(60),
    before_sleep=lambda retry_state: print(
        f"Error occurred: {retry_state.outcome.exception() if retry_state.outcome else 'unkown'}. "
        f"\nWaiting 60 seconds before retry {retry_state.attempt_number}..."
    ),
)
def embed_batch(batch):
    response = client.models.embed_content(
        model="gemini-embedding-001",
        contents=batch,
        config=EmbedContentConfig(
            task_type="SEMANTIC_SIMILARITY",
        ),
    )
    return response


# Process words in batches of 100
batch_size = 100
embeddings = []

for i in range(0, len(words), batch_size):
    batch = words[i : i + batch_size]
    response = embed_batch(batch)
    assert response.embeddings
    for emb in response.embeddings:
        assert emb.values
        embeddings.append(emb.values)
    print(f"Processed batch {i // batch_size + 1}: {len(batch)} words")

print(f"Total embeddings: {len(embeddings)}")

Processed batch 1: 100 words
Processed batch 2: 100 words
Processed batch 3: 100 words
Processed batch 4: 100 words
Processed batch 5: 100 words
Processed batch 6: 8 words
Total embeddings: 508


In [21]:
from itertools import zip_longest

import numpy as np


def cosine_similarity(vec1: list[float], vec2: list[float]) -> float:
    """Calculate cosine similarity between two 1D numpy arrays."""
    arr1 = np.array(vec1)
    arr2 = np.array(vec2)
    norm1 = np.linalg.norm(arr1)
    norm2 = np.linalg.norm(arr2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return np.dot(arr1, arr2) / (norm1 * norm2)


theme_similarity = [cosine_similarity(e, embeddings[0]) for e in embeddings]

df = pd.DataFrame(
    list(zip_longest(words, embeddings, theme_similarity, fillvalue=np.nan)),
    columns=np.array(["word", "embedding", "theme_similarity"]),
).sort_values("theme_similarity", ascending=False)
df.head(40)

,word,embedding,theme_similarity
0,We beg to differ,"[-0.016641049, 0.00031212124, 0.010368948, -0....",1.000000
422,OPPOSITION,"[-0.0052631064, 0.0065230574, 0.012063958, -0....",0.877942
377,BELIE,"[-0.00995308, 0.006791146, 0.013842981, -0.069...",0.857916
265,OPPOSITIONIST,"[-0.0072563286, -0.004622356, 0.018709911, -0....",0.856080
153,OPPOSITIONS,"[-0.0065241386, -0.0054725893, 0.01741129, -0....",0.853830
362,BELIES,"[-0.0061850203, 0.017165095, 0.009829766, -0.0...",0.846051
399,OPPOSITE,"[-0.008894053, 0.0017337105, 0.0036032887, -0....",0.836516
409,OPPOSITES,"[-0.016050741, -0.00027076015, 0.009416437, -0...",0.825826
133,BELIED,"[-0.005177557, 0.0028836736, -0.0069509363, -0...",0.819912
78,DETRACT,"[-0.005979104, 0.0016402858, -0.0026144413, -0...",0.819101


In [24]:
df[df["word"] == "ATTRACT"]

,word,embedding,theme_similarity
420,ATTRACT,"[-0.027488235, -0.0070873527, 0.017679159, -0....",0.78143


In [33]:
response = client.models.embed_content(
    model="gemini-embedding-001",
    contents=["OPPOSITES RIGID"],
    config=EmbedContentConfig(
        task_type="SEMANTIC_SIMILARITY",
    ),
)
assert response.embeddings
emb = response.embeddings[0].values
assert emb
cosine_similarity(emb, embeddings[0])

np.float64(0.7941492961309876)

"OPPOSITES RIGID" (an incorrect spangram) is closer to "We beg to differ" than "OPPOSITIES ATTRACT" (the correct spangram).

So semantic similarity is not a silver bullet for ranking solutions.


In [59]:
from typing import Iterable


@retry(stop=stop_after_attempt(3))
def find_best_spangram(theme: str, candidates: Iterable[str]):
    prompt = f"""
    You are given a cryptic clue and a set of candidate phrases. Your task is to find the phrase that fits the clue best. The phrase should be grammatically correct.

    Answer only with the exact text of the best candidate.

    <clue>{theme}</clue>

    <candidates>
    {"\n".join(f"<candidate>{cand.lower()}</candidate>" for cand in candidates)}
    </candidates>
    """.strip()
    response = client.models.generate_content(
        model="gemini-2.5-flash", contents=[prompt]
    )
    assert response.text
    assert response.usage_metadata
    print(f"Thinking tokens: {response.usage_metadata.thoughts_token_count}")
    for cand in candidates:
        if cand.lower() == response.text.lower():
            return cand
    raise ValueError(
        f"Model did not respond with a valid candidate, received: {response.text!r}"
    )


find_best_spangram(
    "We beg to differ",
    [
        "OPPOSITES RIGID",
        "OPPOSITES ATTRACT",
        "OPPOSITES ENGAGE",
        "OPPOSITES LOVE",
        "OPPOSITES CONTAIN",
    ],
)

Thinking tokens: 2871


'OPPOSITES ATTRACT'

In [ ]:
from strands_solver.grid_coverer import GridCoverer
from strands_solver.word_finder import WordFinder
from strands_solver.solver import Solver
from strands_solver.spangram_finder import SpangramFinder
from strands_solver.main import get_puzzle

puzzle = get_puzzle("2025-12-01")

solver = Solver(
    finder=WordFinder(puzzle.grid),
    coverer=GridCoverer(puzzle.grid),
    spangram_finder=SpangramFinder(puzzle.grid, num_words=puzzle.num_words),
)
solutions = solver.solve()

2025-12-12 20:27:28,088 [INFO] HTTP Request: GET https://www.nytimes.com/svc/strands/v2/2025-12-01.json "HTTP/1.1 200 OK"
2025-12-12 20:27:28,089 [INFO] Loading dictionary
2025-12-12 20:27:28,179 [INFO] Loaded 172822 words
2025-12-12 20:27:28,180 [INFO] Finding words
2025-12-12 20:27:28,207 [INFO] Found 869 words
2025-12-12 20:27:28,207 [INFO] Covering grid
2025-12-12 20:27:32,248 [INFO] Found 2698 covers
2025-12-12 20:27:32,249 [INFO] Finding spangrams
2025-12-12 20:27:34,222 [INFO] Found 3879 solutions with spangrams


In [ ]:
spangrams = set[str]()
for solution in solutions:
    spangram = " ".join(strand.string for strand in solution.spangram)
    spangrams.add(spangram)
len(spangrams)

712

In [ ]:
sorted(spangrams, key=lambda spangram: len(spangram.split()))
# why is HOCKEY RINK not included?

['ZEALS HOCK PUCKA REINK',
 'BODY REINK GOAL HOCK',
 'BODY REINK GOALS HOCK',
 'ROBIN KORE GALS HOCK',
 'ZEALS HOCK AMBO REINK',
 'ZEAL SHOCK PUCKA REINK',
 'REINK ORGY AMBO CONSIST',
 'ROBE GOAL HOCK PUCK',
 'ROBE RAZES HOCK PUCK',
 'TICK SINS COEMBODY REINK',
 'BODY REINK GOAL SHOCK',
 'BOAST SINS COEMBODY REINK',
 'COEMBODY REINK GOAL HOCKS',
 'SHOCK AMBO REINK ORGY',
 'BRINK GOAL HOCK PUCE',
 'ZEKS HOCK PUCES SKAT',
 'COEMBODY REINK GOAL SHOCK',
 'REINK GOALS HOCK PUCK',
 'HOCK AMBO REINK ORGY',
 'REINK ORGY ZEAL HOCKS',
 'COEMBODY REINK GOAL HOCK',
 'HOCK PUCK AMBO REINK',
 'GOAL HOCK PUCES SKAT',
 'HOCK PUCES SKAT NICKS',
 'RICK SINS COEMBODY REINK',
 'SHOCK PUCK AMBO REINK',
 'BODY REINK GOAL HOCKS',
 'REINK GOAL HOCK PUCK',
 'ROBE RINK GOAL HOCKS',
 'BRINK GOAL HOCK PUCES',
 'PUCKA REINK ORGY ZEAL',
 'REINK ORGY ZEAL SHOCK',
 'ROBE GOAL SHOCK PUCK',
 'COEMBODY REINK GOALS HOCK',
 'HOCK PUCKA REINK ORGY',
 'ZEKS HOCK PUCES TASK',
 'ROBE RINK GOALS HOCK',
 'PUCK AMBO REINK ORGY',

In [63]:
find_best_spangram("We beg to differ", spangrams)

Thinking tokens: 1378


'OPPOSITES ATTRACT'

## Embedder playground


In [4]:
from strands_solver.embedder import Embedder

embedder = Embedder()

In [5]:
# Embed some words via API (cached=False)
test_words = ["hello", "world", "python", "embedding"]
result = embedder.get_embeddings(test_words, cached=False)

print(f"Got {len(result)} embeddings")
for word, vec in result.items():
    print(f"  {word}: dim={len(vec)}, first 5 values={vec[:5]}")

Got 4 embeddings
  hello: dim=3072, first 5 values=[-0.03452844, 0.009059422, 0.007681673, -0.057083048, -0.029342607]
  world: dim=3072, first 5 values=[-0.018003238, -0.0058756433, 0.025875887, -0.060153086, -0.017594127]
  python: dim=3072, first 5 values=[-0.0119025335, -0.0009590619, 0.007020803, -0.06769121, -0.019711033]
  embedding: dim=3072, first 5 values=[-0.017193511, 0.006651974, -0.004125737, -0.079775475, -0.00868848]


In [6]:
# Store embeddings in SQLite
embedder.store_embeddings(result)
print("Stored embeddings in DB")

Stored embeddings in DB


In [7]:
# Retrieve from cache (cached=True)
cached = embedder.get_embeddings(["hello", "python"], cached=True)

print("Retrieved from cache:")
for word, vec in cached.items():
    print(f"  {word}: dim={len(vec)}, first 5 values={vec[:5]}")

Retrieved from cache:
  hello: dim=3072, first 5 values=[-0.03452844172716141, 0.009059421718120575, 0.007681672926992178, -0.057083047926425934, -0.02934260666370392]
  python: dim=3072, first 5 values=[-0.01190253347158432, -0.0009590619010850787, 0.007020803168416023, -0.06769120693206787, -0.019711032509803772]


In [8]:
# Try to get missing embedding from cache (raises KeyError)
try:
    embedder.get_embeddings(["not_in_cache"], cached=True)
except KeyError as e:
    print(f"Expected error: {e}")

Expected error: "Embedding not found in cache: 'not_in_cache'"


In [9]:
# Verify cached values match original (roundtrip check)
import numpy as np

for word in ["hello", "python"]:
    original = result[word]
    from_cache = cached[word]
    # float32 serialization loses some precision
    assert np.allclose(original, from_cache, rtol=1e-5)

print("Roundtrip check passed!")

Roundtrip check passed!


In [ ]:
# Close DB connection when done
embedder.close()